In [7]:
from io_utils import read_fbin, read_ibin
import faiss
print(faiss.__version__)
import numpy as np
file_path = "/root/code/vectordbindexing/Text2Image/base.1M.fbin"
query_path = "/root/code/vectordbindexing/Text2Image/query.public.100K.fbin"
query_train_path = "/root/code/vectordbindexing/data/t2i-10M/query.train.10M.fbin"
ground_truth_path = "/root/code/vectordbindexing/Text2Image/groundtruth.public.100K.ibin"

# read datasets
print("\n\nreading image vector: ---")
data_vector = read_fbin(file_path)
print(type(data_vector))
print(data_vector.ndim, data_vector.shape, data_vector.dtype, data_vector.size)
# print(data_vector[:1])  # Print first 1 elements to verify content

train_data_vector = data_vector[:100000]
insert_1_percent = data_vector[500000:505000]
insert_2_percent = data_vector[505000:510000]
insert_3_percent = data_vector[510000:515000]
insert_4_percent = data_vector[515000:520000]
insert_5_percent = data_vector[520000:525000]
insert_10_percent = data_vector[525000:550000]

# read querys
print("\n\nreading querys: ---")
query_vector = read_fbin(query_path)
print(type(query_vector))
print(query_vector.ndim, query_vector.shape, query_vector.dtype, query_vector.size)
# print(query_vector[0])  # Print first 3 elements to verify content

# read train querys
# print("\n\nreading train querys: ---")
# query_train_vector = read_fbin(query_train_path)
# print(type(query_train_vector))
# print(query_train_vector.ndim, query_train_vector.shape, query_train_vector.dtype, query_train_vector.size)
# print(query_train_vector[0])  # Print first 3 elements to verify content


1.11.0


reading image vector: ---


<class 'numpy.ndarray'>
2 (1000000, 200) float32 200000000


reading querys: ---
<class 'numpy.ndarray'>
2 (100000, 200) float32 20000000


In [8]:
import hnsw_cosine_status_high as hnsw_cosine
import simple_sim_hash
import importlib
importlib.reload(hnsw_cosine)

# M=64 比较合适，甚至更宽的宽度
# 这里是个经验值：会在增加宽度的同时，逐渐达到一个稳定值
index = hnsw_cosine.HNSWIndex(M=64, ef_construction=128, ef_search=64, random_seed=1)
simHash = simple_sim_hash.SimpleSimHash(dim=200)

IMAGE_IDX_SET = set()

# 形状 [N,200]（先用1M子集或更小切片做原型）
for img_id, vec in enumerate(train_data_vector):        # 可加 tqdm、批量 flush
    index.add_item_fast10k(vec, lsh=simHash, limit=100)
    IMAGE_IDX_SET.add(img_id)

for qid, vec in enumerate(query_vector):
    index.add_item_fast10k(vec, lsh=simHash, limit=100)

In [3]:
# 读取faiss搜索结果，获取 query_vector 和 search 结果
import json
train_query_list = {}
test_query_list = {}

with open("./TempResults/search_results_100K.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    for query_idx, vec_list in data.items():
        mList = []
        for x in vec_list:
            mList.append(x - int(query_idx))
        if int(query_idx) % 6 != 0:
            train_query_list[int(query_idx)] = mList
        else:
            test_query_list[int(query_idx)] = mList
print(f"num of train: {len(train_query_list)}")
print(f"num of test: {len(test_query_list)}")

num of train: 83333
num of test: 16667


In [9]:
# OOD search steps
NUM_STEPS = []
PHASE_ANALYSIS = []
for qid, target_list in test_query_list.items():
    q = query_vector[qid]
    for target_id in target_list[:10]:
        if target_id not in IMAGE_IDX_SET:
            continue
        # 使用阶段分析功能
        out = index.search_steps_to_target(q, target_id, k=10, ef=64, analyze_phases=True, verbose=False, multi_path_search=True, max_paths=3)
        NUM_STEPS.append(len(out["trace"]))
        if "phase_analysis" in out:
            PHASE_ANALYSIS.append(out["phase_analysis"])


# 分析阶段统计
if PHASE_ANALYSIS:
    print("\n=== 阶段分析统计 ===")
    phase_1_steps = [pa["phase_1"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_2_steps = [pa["phase_2"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_1_accel_edges = [pa["phase_1"]["accel_edges"] for pa in PHASE_ANALYSIS]
    phase_2_accel_edges = [pa["phase_2"]["accel_edges"] for pa in PHASE_ANALYSIS]
    
    print(f"第一阶段 (快速靠近) - 平均步数: {np.mean(phase_1_steps):.2f}, 平均加速边: {np.mean(phase_1_accel_edges):.2f}")
    print(f"第二阶段 (Beam Search) - 平均步数: {np.mean(phase_2_steps):.2f}, 平均加速边: {np.mean(phase_2_accel_edges):.2f}")
    
    # 计算加速边使用比例
    total_accel_edges = [pa["total_accel_edges"] for pa in PHASE_ANALYSIS]
    total_steps = [pa["total_steps"] for pa in PHASE_ANALYSIS]
    accel_edge_ratios = [accel/steps if steps > 0 else 0 for accel, steps in zip(total_accel_edges, total_steps)]
    
    print(f"整体加速边使用比例: {np.mean(accel_edge_ratios):.2%}")
    
    # 分析哪些查询受益最多
    if len(PHASE_ANALYSIS) > 0:
        best_benefit_idx = np.argmax(accel_edge_ratios)
        best_benefit = PHASE_ANALYSIS[best_benefit_idx]
        print(f"\n加速边受益最多的查询:")
        print(f"  第一阶段: {best_benefit['phase_1']['step_count']} 步, {best_benefit['phase_1']['accel_edges']} 条加速边")
        print(f"  第二阶段: {best_benefit['phase_2']['step_count']} 步, {best_benefit['phase_2']['accel_edges']} 条加速边")
        print(f"  总步数: {best_benefit['total_steps']}, 总加速边: {best_benefit['total_accel_edges']}")
        print(f"  加速边比例: {best_benefit['overall_accel_edge_ratio']:.2%}")


arr_ori_bak = np.array(NUM_STEPS, dtype=np.float64)
arr_ori = arr_ori_bak.copy()
arr_ori.sort()

mean_steps = arr_ori.mean()
P50_steps = np.percentile(arr_ori, 50)
p99_steps = np.percentile(arr_ori, 99)
print(f"\n原始搜索统计:")
print(f"mean steps: {mean_steps}")
print(f"middle steps: {P50_steps}")
print(f"p99 steps: {p99_steps}")


=== 阶段分析统计 ===
第一阶段 (快速靠近) - 平均步数: 406.25, 平均加速边: 0.00
第二阶段 (Beam Search) - 平均步数: 72.71, 平均加速边: 0.00
整体加速边使用比例: 0.00%

加速边受益最多的查询:
  第一阶段: 412 步, 0 条加速边
  第二阶段: 110 步, 0 条加速边
  总步数: 522, 总加速边: 0
  加速边比例: 0.00%

原始搜索统计:
mean steps: 478.96458644371455
middle steps: 460.0
p99 steps: 906.0


In [10]:
# 使用新的 RoarGraph 风格的 cross distribution 边构建
print("\n=== 构建 RoarGraph 风格的 Cross Distribution 边 ===")
for query in query_vector:
    stats = index.build_cross_distribution_edges(
        max_new_edges_per_node=4,
        query=query,
    )
print("Cross distribution 边构建统计:")
print(stats)

# 获取 cross distribution 边的统计信息
cross_stats = index.get_cross_distribution_stats()
print("\nCross distribution 边统计:")
print(f"总添加的 cross distribution 边: {cross_stats['total_cross_edges']}")
print(f"被删除的 cross distribution 边: {cross_stats['deleted_cross_edges']}")
print(f"活跃的 cross distribution 边: {cross_stats['active_cross_edges']}")



=== 构建 RoarGraph 风格的 Cross Distribution 边 ===


Cross distribution 边构建统计:
{'query_processed': True, 'layer_1_nodes_total': 3092, 'top_k_selected': 10, 'pairs_considered': 45, 'pairs_added': 10, 'skipped_existing': 35, 'pruned_by_cap': 0, 'edges_added': 10, 'top_k_nodes': [123656, 125677, 158436, 183136, 132312, 179175, 141094, 190128, 156261, 174419], 'query_distance': 0.3494441509246826}

Cross distribution 边统计:
总添加的 cross distribution 边: 1512998
被删除的 cross distribution 边: 1374116
活跃的 cross distribution 边: 138882


In [11]:
# OOD search steps
NUM_STEPS = []
PHASE_ANALYSIS = []
for qid, target_list in test_query_list.items():
    q = query_vector[qid]
    for target_id in target_list[:10]:
        if target_id not in IMAGE_IDX_SET:
            continue
        # 使用阶段分析功能
        out = index.search_steps_to_target(q, target_id, k=10, ef=64, analyze_phases=True, verbose=False, multi_path_search=True, max_paths=3)
        NUM_STEPS.append(len(out["trace"]))
        if "phase_analysis" in out:
            PHASE_ANALYSIS.append(out["phase_analysis"])


# 分析阶段统计
if PHASE_ANALYSIS:
    print("\n=== 阶段分析统计 ===")
    phase_1_steps = [pa["phase_1"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_2_steps = [pa["phase_2"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_1_accel_edges = [pa["phase_1"]["accel_edges"] for pa in PHASE_ANALYSIS]
    phase_2_accel_edges = [pa["phase_2"]["accel_edges"] for pa in PHASE_ANALYSIS]
    
    print(f"第一阶段 (快速靠近) - 平均步数: {np.mean(phase_1_steps):.2f}, 平均加速边: {np.mean(phase_1_accel_edges):.2f}")
    print(f"第二阶段 (Beam Search) - 平均步数: {np.mean(phase_2_steps):.2f}, 平均加速边: {np.mean(phase_2_accel_edges):.2f}")
    
    # 计算加速边使用比例
    total_accel_edges = [pa["total_accel_edges"] for pa in PHASE_ANALYSIS]
    total_steps = [pa["total_steps"] for pa in PHASE_ANALYSIS]
    accel_edge_ratios = [accel/steps if steps > 0 else 0 for accel, steps in zip(total_accel_edges, total_steps)]
    
    print(f"整体加速边使用比例: {np.mean(accel_edge_ratios):.2%}")
    
    # 分析哪些查询受益最多
    if len(PHASE_ANALYSIS) > 0:
        best_benefit_idx = np.argmax(accel_edge_ratios)
        best_benefit = PHASE_ANALYSIS[best_benefit_idx]
        print(f"\n加速边受益最多的查询:")
        print(f"  第一阶段: {best_benefit['phase_1']['step_count']} 步, {best_benefit['phase_1']['accel_edges']} 条加速边")
        print(f"  第二阶段: {best_benefit['phase_2']['step_count']} 步, {best_benefit['phase_2']['accel_edges']} 条加速边")
        print(f"  总步数: {best_benefit['total_steps']}, 总加速边: {best_benefit['total_accel_edges']}")
        print(f"  加速边比例: {best_benefit['overall_accel_edge_ratio']:.2%}")


arr_ori_bak = np.array(NUM_STEPS, dtype=np.float64)
arr_ori = arr_ori_bak.copy()
arr_ori.sort()

mean_steps = arr_ori.mean()
P50_steps = np.percentile(arr_ori, 50)
p99_steps = np.percentile(arr_ori, 99)
print(f"\n原始搜索统计:")
print(f"mean steps: {mean_steps}")
print(f"middle steps: {P50_steps}")
print(f"p99 steps: {p99_steps}")


=== 阶段分析统计 ===
第一阶段 (快速靠近) - 平均步数: 525.21, 平均加速边: 275.88
第二阶段 (Beam Search) - 平均步数: 72.71, 平均加速边: 0.00
整体加速边使用比例: 47.27%

加速边受益最多的查询:
  第一阶段: 372 步, 267 条加速边
  第二阶段: 0 步, 0 条加速边
  总步数: 372, 总加速边: 267
  加速边比例: 71.77%

原始搜索统计:
mean steps: 597.9242759032547
middle steps: 585.0
p99 steps: 1040.0
